<h3>Table prep</h3>

In [1]:
import pandas as pd
import sqlite3
import numpy as np

con = sqlite3.connect('soccer.sqlite')
df_train = {}

In [16]:
player_df = pd.read_sql_query('SELECT * FROM player', db)
df_train['player'] = player_df.sample(n=100)
df_train['player'].head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
7110,7121,206743,Matias Jones,202424,1991-07-01 00:00:00,175.26,165
10909,10925,40731,"Yazid Mansouri,30",7384,1978-02-25 00:00:00,175.26,152
9773,9789,281993,Simmy Nwankwo,219496,1992-05-07 00:00:00,195.58,176
7118,7129,206322,Matias Vecino,219985,1991-08-24 00:00:00,187.96,179
8935,8949,212645,Riccardo Saponara,201046,1991-12-21 00:00:00,182.88,161


In [17]:
player_attr_df = pd.read_sql_query('SELECT * FROM player_attributes', db)

In [18]:
boolean_series = player_attr_df.player_api_id.isin(df_train['player'].player_api_id)
df_train['player_attributes'] = player_attr_df[boolean_series]
df_train['player_attributes'].head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
5060,5061,178452,39302,2016-05-05 00:00:00,67.0,67.0,right,medium,medium,12.0,...,25.0,22.0,11.0,15.0,20.0,68.0,68.0,61.0,65.0,72.0
5061,5062,178452,39302,2016-04-21 00:00:00,65.0,65.0,right,medium,medium,12.0,...,25.0,22.0,11.0,15.0,20.0,63.0,64.0,61.0,65.0,69.0
5062,5063,178452,39302,2016-03-10 00:00:00,65.0,66.0,right,medium,medium,12.0,...,25.0,22.0,11.0,15.0,20.0,63.0,64.0,61.0,65.0,69.0
5063,5064,178452,39302,2015-11-06 00:00:00,65.0,67.0,right,medium,medium,12.0,...,25.0,22.0,11.0,15.0,20.0,63.0,64.0,61.0,65.0,69.0
5064,5065,178452,39302,2015-09-21 00:00:00,65.0,68.0,right,medium,medium,12.0,...,25.0,22.0,11.0,15.0,20.0,63.0,64.0,61.0,65.0,69.0


In [19]:
for k, v in df_train.items():
    df_train[k] = df_train[k].replace([np.inf, -np.inf], np.nan)
    df_train[k] = df_train[k].dropna()

In [20]:
df_train['player'][df_train['player'].isna().any(axis=1)]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight


In [21]:
df_train['player_attributes'][df_train['player_attributes'].isna().any(axis=1)]

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes


<h3>Starting SDV</h3>

In [2]:
from sdv import Metadata

metadata = Metadata()
metadata

Metadata
  root_path: .
  tables: []
  relationships:

In [23]:
metadata.add_table(
    name='player',
    data=df_train['player'],
    primary_key='id'
)

In [24]:
metadata.add_table(
    name='player_attributes',
    data=df_train['player_attributes'],
    primary_key='id',
    parent='player',
    foreign_key='player_api_id'
)

In [25]:
metadata.to_dict()

{'tables': {'player': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'player_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'player_name': {'type': 'categorical'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'birthday': {'type': 'categorical'},
    'height': {'type': 'numerical', 'subtype': 'float'},
    'weight': {'type': 'numerical', 'subtype': 'integer'}},
   'primary_key': 'id'},
  'player_attributes': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'player_api_id': {'type': 'id',
     'subtype': 'integer',
     'ref': {'table': 'player', 'field': 'id'}},
    'date': {'type': 'categorical'},
    'overall_rating': {'type': 'numerical', 'subtype': 'float'},
    'potential': {'type': 'numerical', 'subtype': 'float'},
    'preferred_foot': {'type': 'categorical'},
    'attacking_work_rate': {'type': 'categorical'},
    'defensive_work_rate': {'type':

In [26]:
from sdv.relational import HMA1

model = HMA1(metadata)
model.fit(df_train)

In [27]:
new_data = model.sample()

ValueError: array must not contain infs or NaNs

In [16]:
def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

def get_table_order(db):
    '''This method returns the order of tables to be generated. Tables have relations between each other (FK) that needs to be created first in order to generate consistend data. 
    TODO: Remove pk; fk stuff from here. Method already big enough, will get own method to call when needed. '''
    rows = db.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
    tables = [row[0].lower() for row in rows]
    
    if "sqlite_sequence" in tables:
        tables.remove('sqlite_sequence')
    
    fk_relation = {}
    pk_relation = {}
    
    table_order = []
    
    for table in tables:
        rows = db.execute("PRAGMA table_info({})".format(sql_identifier(table)))
        attributes = rows.fetchall()
        
        pk = []
        for attr in attributes:
            if attr[5] == 1:
                pk.append(attr[1])    
        pk_relation[table] = pk
        
        rows = db.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
        foreign_key_list = rows.fetchall()
        fkeys = []
        for fk in foreign_key_list:
            fkeys.append({'table': fk[2].lower(), 'origin': fk[3], 'dest': fk[4]})
        fk_relation[table] = fkeys
        
        if not fkeys:
            table_order.append(table)
        else:
            contained = True
            depends = []
            for fk in fkeys:
                if fk['table'] not in depends:
                    depends.append(fk['table'])
                if fk['table'] not in table_order:
                    contained = False
            if contained:
                #table_order.append((table, depends))
                table_order.append(table)
                
                
    maxLoop = 5
    while len(table_order) <= len(tables):
        for table, fkeys in fk_relation.items():
            if table in table_order:
                continue
                
            contained = True
            depends = []
            for fk in fkeys:
                if fk['table'] not in depends:
                    depends.append(fk['table'])
                if fk['table'] not in table_order:
                    contained = False
            if contained:
                table_order.append(table)  
                
        maxLoop -= 1
        if maxLoop <= 0:
            break
        
    return (table_order, pk_relation, fk_relation)


In [32]:
con = sqlite3.connect('E:\GitHub Repos\TensorFlow training\SoccerDataset\database.sqlite')
table_order, pk_relation, fk_relation = get_table_order(con)

In [33]:
pk_relation

{'player_attributes': ['id'],
 'player': ['id'],
 'match': ['id'],
 'league': ['id'],
 'country': ['id'],
 'team': ['id'],
 'team_attributes': ['id']}

In [34]:
fk_relation

{'player_attributes': [{'table': 'player',
   'origin': 'player_api_id',
   'dest': 'player_api_id'},
  {'table': 'player',
   'origin': 'player_fifa_api_id',
   'dest': 'player_fifa_api_id'}],
 'player': [],
 'match': [{'table': 'player',
   'origin': 'away_player_11',
   'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_10', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_9', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_8', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_7', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_6', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_5', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_4', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_3', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_2', 'dest': 'player_api_id'},
  {'table': 'player', 'o

In [39]:
def get_Metadata(con, tables, pk_relation, fk_relation):
    metadata = Metadata()

    for table in tables:
        df = pd.read_sql_query('SELECT * FROM ' + table + ' LIMIT 5', con)

        metadata.add_table(
            name=table,
            data=df,
            primary_key=pk_relation[table][0] #TODO: Könnte sein, dass es mehr prim keys gibt. Dann schauen.
        )

    return metadata

In [40]:
metadata = get_Metadata(con, table_order, pk_relation, fk_relation)

In [41]:
metadata.to_dict()

{'tables': {'player': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'player_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'player_name': {'type': 'categorical'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'birthday': {'type': 'categorical'},
    'height': {'type': 'numerical', 'subtype': 'float'},
    'weight': {'type': 'numerical', 'subtype': 'integer'}},
   'primary_key': 'id'},
  'country': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'name': {'type': 'categorical'}},
   'primary_key': 'id'},
  'team': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'team_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'team_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'team_long_name': {'type': 'categorical'},
    'team_short_name': {'type': 'categorical'}},
   'primary_key': 'id'},
  'team_attributes': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'team_fifa_api_id': {'typ

In [25]:
df = pd.read_sql_query('SELECT * FROM ' + table_order[0], con)

In [43]:
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')